# An introduction to PyMC3 & exoplanet for astronomers

By: **Dan Foreman-Mackey**

In this tutorial, we will learn how to use [exoplanet](https://exoplanet.dfm.io) and [PyMC3](https://docs.pymc.io) to do Markov chain Monte Carlo (MCMC) with a focus on fitting [TESS](https://en.wikipedia.org/wiki/Transiting_Exoplanet_Survey_Satellite) data.
But first, we have to do some setup:

In [ ]:
# We want to see plots in the browser
%matplotlib inline

In [ ]:
# This is the one dependency missing on the science platform
!pip install -q corner

# Let's make the plots look a little nicer
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["figure.dpi"] = 100
rcParams["font.size"] = 16

# The installation of Theano is a little broken (but it'll work
# fine for our purposes). Deal with those issues as follows:
import os
import warnings
os.environ["MKL_THREADING_LAYER"] = "GNU"
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## Part 1: Fitting a line to data with PyMC3

In this example, we'll do the classic example of fitting a line to data using PyMC3.
But it's not just any data!
Instead, we'll fit a (very simplified) probabilistic mass-radius relationship to the sub-Neptune planets listed on the Exoplanet Archive.
You can take a look at [Wolfgang, Rogers & Ford (2016)](https://arxiv.org/abs/1504.07557) or [Chen & Kipping (2016)](https://arxiv.org/abs/1603.08614) for more literature on this topic, but today we'll fit a pretty simple model.

For small ($1\,R_\oplus < R < 4\,R_\oplus$) planets with both mass and radius measurements, we'll fit the relation:

$$
\log_{10} M / M_\oplus = A\,\log_{10} R / R_\oplus + B
$$

which turns out to be the equation for a line.
For simplicity, we'll ignore the uncertainties on radius (although it wouldn't be too hard to incorporate those: [ref](https://dfm.io/posts/fitting-a-plane/)), but we will allow for this relation to have some intrinsic width with standard deviation $\Sigma_M$ in the mass direction.
Under these assumptions, the likelihood function for the $n$-th is:

$$
\log_{10} M_n / M_\oplus \sim \mathrm{Normal}\left(A\,\log_{10} R_n / R_\oplus + B,\,\sqrt{{\sigma_n}^2 + {\Sigma_M}^2} \right)
$$

where ${\sigma_n}^2$ is the reported uncertainty on the log-mass of the $n$-th planet.
Using this model, we'll fit for $A$, $B$, and $\Sigma_M$ using PyMC3.

But first, we need to download and format the data:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

url = "https://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=planets&select=*"
data = pd.read_csv(url)

selected = (1 < data.pl_rade) & (data.pl_rade < 4)
selected &= data.pl_masse + data.pl_masseerr2 > 0
selected &= data.pl_rade + data.pl_radeerr2 > 0
for column in ["pl_rade", "pl_radeerr1", "pl_radeerr2", "pl_masse", "pl_masseerr1", "pl_masseerr2"]:
    selected &= np.isfinite(data[column]) & (np.abs(data[column]) > 0)
data = pd.DataFrame(data[selected])
    
log_radius = np.log10(np.array(data.pl_rade))
log_radius_err = 0.5 * np.array(np.log10(data.pl_rade+data.pl_radeerr1) - np.log10(data.pl_rade+data.pl_radeerr2))
log_mass = np.log10(np.array(data.pl_masse))
log_mass_err = 0.5 * np.array(np.log10(data.pl_masse+data.pl_masseerr1) - np.log10(data.pl_masse+data.pl_masseerr2))

plt.errorbar(log_radius, log_mass, xerr=log_radius_err, yerr=log_mass_err, fmt=".k")
plt.xlabel("log10(radius / Earth radius)")
plt.ylabel("log10(mass / Earth mass)");

To fit a model to these data, our model will have 3 parameters: the slope $A$, the intercept $B$, and the log of the line width $\log(\Sigma_M)$.
To start, let's choose broad uniform priors on these parameters:

$$
\begin{eqnarray}
p(A) &=& \left\{\begin{array}{ll}
1/10 & \mathrm{if}\,-5 < A < 5 \\
0 & \mathrm{otherwise} \\
\end{array}\right. \\
p(B) &=& \left\{\begin{array}{ll}
1/10 & \mathrm{if}\,-5 < B < 5 \\
0 & \mathrm{otherwise} \\
\end{array}\right. \\
p(\log(\Sigma_M)) &=& \left\{\begin{array}{ll}
1/10 & \mathrm{if}\,-5 < \log(\Sigma_M) < 5 \\
0 & \mathrm{otherwise} \\
\end{array}\right.
\end{eqnarray}
$$

Then, the log-likelihood function will be

$$
\log p(\{M_n\}\,|\,A,\,B,\,\log(\Sigma_M)) = -\frac{1}{2}\sum_{n=1}^N \left[\frac{(\log_{10} M_n/M_\oplus - A\,\log_{10}R_n/R_\oplus - B)^2}{{\Sigma_M}^2 + {\sigma_n}^2} + \log(2\,\pi\,({\Sigma_M}^2 + {\sigma_n}^2))\right]
$$

[**Note:** the second normalization term is needed in this model because we are fitting for $\Sigma_M$ and the second term is *not* a constant.]

Another way of writing this model that might not be familiar is the following:

$$
\begin{eqnarray}
A &\sim& \mathrm{Uniform}(-5,\,5) \\
B &\sim& \mathrm{Uniform}(-5,\,5) \\
\log(\sigma) &\sim& \mathrm{Uniform}(-5,\,5) \\
\log_{10} M_n/M_\oplus &\sim& \mathrm{Normal}\left(A\,\log_{10}R_n/R_\oplus + B,\,\sqrt{{\Sigma_M}^2 + {\sigma_n}^2}\right)
\end{eqnarray}
$$

This is the way that a model like this is often defined in statistics and it will be useful when we implement out model in PyMC3 so take a moment to make sure that you understand the notation.

Now, let's implement this model in PyMC3.
The documentation for the distributions available in PyMC3's modeling language can be [found here](https://docs.pymc.io/api/distributions/continuous.html) and these will come in handy as you write your model.

In [ ]:
import pymc3 as pm

with pm.Model() as model:
    
    # Define the priors on each parameter. For example, the
    # definition of "A" will look something like the following:
    A = pm.Uniform("A", lower=-5, upper=5)
    
    # Based on the definition of "A", define "B" and "logS" here:
    # Note: for the following cells to work, you must call these
    # variables "B" and "logS" exactly.
    
    # YOUR CODE GOES HERE...
    
    # Define the likelihood below this line. A few tips:
    #  1. For mathematical operations like "exp", you can't use
    #     numpy. Instead, use the mathematical operations defined
    #     in "pm.math".
    #  2. To condition on data, you use the "observed" keyword
    #     argument to any distribution. In this case, we want to
    #     use the "Normal" distribution (look up the docs for
    #     this).
    
    # YOUR CODE GOES HERE...
    
    # This is how you will sample the model. Take a look at the
    # docs to see that other parameters that are available.
    trace = pm.sample(draws=2000, tune=2000, target_accept=0.9)

Now since we now have samples, let's make some diagnostic plots.
The first plot to look at is the "traceplot" implemented in PyMC3.
In this plot, you'll see the marginalized distribution for each parameter on the left and the trace plot (parameter value as a function of step number) on the right.
In each panel, you should see at least two lines with different colors.
These are the results of different independent chains and if the results are substantially different in the different chains then there is probably something going wrong.

In [ ]:
pm.traceplot(trace, varnames=["A", "B", "logS"]);

It's also good to quantify that "looking substantially different" argument.
This is implemented in PyMC3 as the "summary" function.
In this table, some of the key columns to look at are `n_eff` and `Rhat`.
* `n_eff` shows an estimate of the number of effective (or independent) samples for that parameter. In this case, `n_eff` should probably be around 500 per chain (there should have been at least 2 chains run depending on your computer).
* `Rhat` shows the [Gelman–Rubin statistic](https://docs.pymc.io/api/diagnostics.html#pymc3.diagnostics.gelman_rubin) and it should be close to 1.

In [ ]:
pm.summary(trace, varnames=["A", "B", "logS"])

The last diagnostic plot that we'll make here is the [corner plot made using corner.py](https://corner.readthedocs.io).
The easiest way to do this using PyMC3 is to first convert the trace to a [Pandas DataFrame](https://pandas.pydata.org/) and then pass that to `corner.py`.
In this plot, we'll compare to the results from [Wolfgang, Rogers & Ford (2016)](https://arxiv.org/abs/1504.07557) where they fit a similar dataset.
This should be qualitatively consistent. If not, try to debug your model to see what went wrong.

In [ ]:
import corner  # https://corner.readthedocs.io
samples = pm.trace_to_dataframe(trace, varnames=["A", "B", "logS"])
corner.corner(samples, truths=[1.3, np.log10(2.7), None]);

It's also useful to look at the predictions of the model in the data space.
In other words: what mass-radius relation does our fit suggest?
In most applications it's often easiest to track these predictions using `pm.Deterministic` nodes in the model, but that would make things more complicated for this example, so we'll compute the predictions directly using the values from the chain.

In [ ]:
# Plot the data
plt.errorbar(log_radius, log_mass, xerr=log_radius_err, yerr=log_mass_err, fmt=".k", label="data")

# Compute the predicted mass-radius relation for each sample
x_pred = np.linspace(-0.1, 0.8, 5000)
lines = trace["A"][:, None] * x_pred[None, :] + trace["B"][:, None]

# Compute percentiles of the prediction
mu_pred = np.median(lines, axis=0)
mn = lines - np.exp(trace["logS"])[:, None]
mn_pred = np.percentile(mn, 16, axis=0)
mx = lines + np.exp(trace["logS"])[:, None]
mx_pred = np.percentile(mx, 84, axis=0)

# Plot the prediction
plt.fill_between(x_pred, mn_pred, mx_pred, color="C0", alpha=0.5)
plt.plot(x_pred, mu_pred, color="C0", label="predicted")

# Format the plot
plt.legend(fontsize=12)
plt.xlabel("log10(radius / Earth radius)")
plt.ylabel("log10(mass / Earth mass)")
plt.xlim(x_pred[0], x_pred[-1]);

**Extra credit:** Here are a few suggestions for things to try out while getting more familiar with PyMC3:

1. Try initializing the parameters using the `testval` argument to the distributions. Does this improve performance in this case? It will substantially improve performance in more complicated examples.
2. Try changing the priors on the parameters. For example, try the "uninformative" prior [recommended by Jake VanderPlas on his blog](http://jakevdp.github.io/blog/2014/06/14/frequentism-and-bayesianism-4-bayesian-in-python/#Prior-on-Slope-and-Intercept).
3. Try re-running your code using a `pm.Deterministic` node to track the value of $\Sigma_M$ itself. This isn't so useful in this case, but you'll see that deterministic nodes can come in very handy later.
4. Try including a mixture model to handle outliers. See [my blog post](https://dfm.io/posts/mixture-models/) for more info about what this model looks like.
5. Incorporate the uncertainties on radius. This can be handled by [analytically marginalizing over the "true" radius and mass](https://dfm.io/posts/fitting-a-plane/) or by directly fitting for the true mass and radius of each planet by including these as parameters in the PyMC3 model. The benefit of using PyMC3 is that it should be able to handle large numbers of parameters for exactly this type of problem.

**Continue to the next part of the tutorial:** [Part 2: Fitting a transit using exoplanet](02_transit.ipynb)